Avaliação de Métricas com Rag-receita
Este notebook demonstra como avaliar métricas utilizando o código do repositório rag-receita.
Aqui você encontrará as etapas para importar as bibliotecas, definir funções auxiliares, executar a avaliação e, opcionalmente, consolidar os resultados e converter CSV para JSON.



Conversão de CSV para JSON (Opcional)
Nesta seção, convertemos um arquivo CSV em JSON, ignorando as linhas sem resposta na coluna especificada.

In [3]:
import csv
import os
import json

def csv_to_json(csv_path, json_path):
    data_list = []
    with open(csv_path, mode='r', encoding='utf-8') as f_in:
        reader = csv.DictReader(f_in)
        for row in reader:
            # Ignorar se não tiver resposta na coluna especificada
            candidate_value = row.get("chatgpt(4o/4mini)+search", "")
            if not candidate_value.strip():
                continue
            data_list.append(row)

    with open(json_path, mode='w', encoding='utf-8') as f_out:
        json.dump(data_list, f_out, indent=2, ensure_ascii=False)

# Defina os caminhos para o arquivo CSV e para o arquivo JSON de saída (ajuste conforme necessário)
csv_path = "files/data.csv"
json_path = "files/data.json"

if not os.path.isfile(csv_path):
    print(f"Arquivo CSV não encontrado em {csv_path}")
else:
    csv_to_json(csv_path, json_path)
    print(f"Conversão concluída! JSON criado em {json_path}")


Conversão concluída! JSON criado em files/data2.json


In [2]:
# Importações básicas e configurações iniciais
import os
import json
import argparse
import asyncio
from dotenv import load_dotenv

# Importações específicas do repositório
from ragas.dataset_schema import SingleTurnSample
from ragas.metrics import (
    ResponseRelevancy,
    FactualCorrectness,
    SemanticSimilarity,
    BleuScore,
    RougeScore
)
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Carrega variáveis de ambiente
load_dotenv()

# Definição das combinações de ROUGE
rouge_combinations = {
    "rouge1": ["precision", "recall", "fmeasure"],
    "rouge2": ["precision", "recall", "fmeasure"],
    "rougeL": ["precision", "recall", "fmeasure"],
}


Funções Auxiliares
Nesta seção, definimos as funções para inicializar as somas das métricas e para calcular a média dos resultados obtidos.

In [3]:
def init_metric_sums(selected_metrics):
    sums = {}
    for m in selected_metrics:
        if m == "rouge":
            # Adiciona submétricas de rouge
            for rtype, modes in rouge_combinations.items():
                for mode in modes:
                    key = f"{rtype}_{mode}"
                    sums[key] = 0.0
        else:
            sums[m] = 0.0
    return sums

def average_metric_sums(metric_sums, count_items):
    if count_items == 0:
        return {k: 0.0 for k in metric_sums}
    else:
        return {k: v / count_items for k, v in metric_sums.items()}


Função de Avaliação de Métricas
Nesta seção, definimos a função assíncrona evaluate_metrics que processa cada item do dataset, calcula as métricas definidas e salva os resultados individualmente e de forma consolidada.

In [4]:
import nest_asyncio
nest_asyncio.apply()

async def evaluate_metrics(
        data,
        question_col="question_text",
        reference_col="answer",
        candidate_col="candidate",
        metrics=None,
        llm_model="gpt-4o-mini",
        embedding_model="text-embedding-3-small",
        output_json="all_results.json"
):
    results_dir = "results_notebook"
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    evaluator_llm = LangchainLLMWrapper(
        ChatOpenAI(model=llm_model, temperature=0)
    )
    evaluator_embedding = OpenAIEmbeddings(model=embedding_model)
    embeddings_wrapper = LangchainEmbeddingsWrapper(evaluator_embedding)

    scorer_answer_relevancy = ResponseRelevancy(llm=evaluator_llm, embeddings=embeddings_wrapper)
    scorer_answer_correctness = FactualCorrectness(llm=evaluator_llm)
    scorer_semantic_similarity = SemanticSimilarity(embeddings=embeddings_wrapper)
    bleu_scorer = BleuScore()

    metric_sums = init_metric_sums(metrics)
    count_items = 0
    total_items = len(data)
    item_results = []

    for idx, item in enumerate(data, start=1):
        q_number = item.get("question_number", str(idx))
        q_text = item.get("question_text", item.get(question_col, ""))

        reference = item.get(reference_col, "")
        candidate = item.get(candidate_col, "")

        if not str(candidate).strip():
            print(f"[{idx}/{total_items}] Questão {q_number} ignorada (candidate vazio).")
            continue

        sample = SingleTurnSample(
            user_input=q_text,
            response=str(candidate),
            reference=str(reference)
        )

        result_item = {
            "question_number": q_number,
            "question_text": q_text,
            "reference": reference,
            "candidate": candidate
        }

        if "answer_relevancy" in metrics:
            relevancy_score = await scorer_answer_relevancy.single_turn_ascore(sample)
            result_item["answer_relevancy"] = relevancy_score
            metric_sums["answer_relevancy"] += relevancy_score

        if "answer_correctness" in metrics:
            correctness_score = await scorer_answer_correctness.single_turn_ascore(sample)
            result_item["answer_correctness"] = correctness_score
            metric_sums["answer_correctness"] += correctness_score

        if "semantic_similarity" in metrics:
            semsim_score = await scorer_semantic_similarity.single_turn_ascore(sample)
            result_item["semantic_similarity"] = semsim_score
            metric_sums["semantic_similarity"] += semsim_score

        if "bleu" in metrics:
            bleu_score_val = await bleu_scorer.single_turn_ascore(sample)
            result_item["bleu"] = bleu_score_val
            metric_sums["bleu"] += bleu_score_val

        if "rouge" in metrics:
            for rouge_type, modes in rouge_combinations.items():
                for mode in modes:
                    scorer = RougeScore(rouge_type=rouge_type, mode=mode)
                    rouge_val = await scorer.single_turn_ascore(sample)
                    column_name = f"{rouge_type}_{mode}"
                    result_item[column_name] = rouge_val
                    metric_sums[column_name] += rouge_val

        item_results.append(result_item)
        count_items += 1

        individual_filename = os.path.join(results_dir, f"answer_{q_number}.json")
        with open(individual_filename, "w", encoding="utf-8") as f:
            json.dump(result_item, f, indent=2, ensure_ascii=False)

        print(f"[{idx}/{total_items}] Questão {q_number} processada e salva em {individual_filename}.", flush=True)

    final_scores = average_metric_sums(metric_sums, count_items)
    output_data = {
        "item_results": item_results,
        "overall_scores": final_scores
    }

    all_results_filename = os.path.join(results_dir, output_json)
    with open(all_results_filename, "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)

    print(f"Todos os resultados foram salvos em {all_results_filename}.")
    return output_data


Execução da Avaliação de Métricas
Nesta seção, definimos os parâmetros, carregamos os dados (arquivo JSON) e executamos a função evaluate_metrics para processar o dataset.

In [5]:
# Defina os parâmetros (ajuste os caminhos e modelos conforme necessário)
filename = "files/data.json"  # Atualize para o caminho correto do seu arquivo que contém as respostas
reference_column = "answer"
candidate_column = "chatgpt(4o/4mini)+search"
question_column = "question_text"
metrics = ["answer_relevancy", "answer_correctness", "semantic_similarity", "bleu", "rouge"]
llm_model = "gpt-4o-mini" # Modelo de LLM a ser utilizado
embedding_model = "text-embedding-3-small" # ou 'text-embedding-3-large' / 'text-embedding-ada-002'
output_json = "notebook_results.json"

# Carregar os dados do arquivo JSON
with open(filename, "r", encoding="utf-8") as f:
    data = json.load(f)

# Executar a avaliação de métricas
results = asyncio.run(evaluate_metrics(
    data=data,
    question_col=question_column,
    reference_col=reference_column,
    candidate_col=candidate_column,
    metrics=metrics,
    llm_model=llm_model,
    embedding_model=embedding_model,
    output_json=output_json
))

print("\n=== MÉTRICAS GLOBAIS ===")
print(json.dumps(results["overall_scores"], indent=2, ensure_ascii=False))


[1/200] Questão 1 processada e salva em results_notebook\answer_1.json.



KeyboardInterrupt



Consolidação dos Resultados (Usar apenas se necessário, o código anterior já salva os resultados.)
Nesta seção, consolidamos os arquivos individuais answer_*.json salvos na pasta results, calculando a média das métricas e gerando o arquivo all_results.json.

In [ ]:
import glob

results_dir = "results"  # Ajuste se necessário
file_pattern = os.path.join(results_dir, "answer_*.json")
file_list = sorted(
    glob.glob(file_pattern),
    key=lambda x: int(os.path.splitext(os.path.basename(x))[0].split('_')[1])
)

if not file_list:
    print("Nenhum arquivo answer_*.json encontrado em", results_dir)
else:
    item_results = []
    non_metric_keys = {"question_number", "question_text", "reference", "candidate"}
    metric_sums = {}
    count_items = 0

    for file in file_list:
        with open(file, "r", encoding="utf-8") as f:
            data_item = json.load(f)
            item_results.append(data_item)
            count_items += 1
            for key, value in data_item.items():
                if key not in non_metric_keys and isinstance(value, (int, float)):
                    metric_sums[key] = metric_sums.get(key, 0.0) + value

    overall_scores = average_metric_sums(metric_sums, count_items)
    output_data = {
        "item_results": item_results,
        "overall_scores": overall_scores
    }

    output_filename = os.path.join(results_dir, "all_results.json")
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(output_data, f, indent=2, ensure_ascii=False)

    print(f"Todos os resultados foram salvos em {output_filename}.", flush=True)
